In [ ]:
import pandas as pd
from tqdm.std import tqdm
import json

## load data

In [ ]:
file_name = 'eye_new_annotation_v2_bm25.csv' ## change it to your own path

In [ ]:
df = pd.read_csv(file_name)

In [ ]:
df

In [ ]:
## extract valid colums
df = df[["Element","GoldName","GoldID","allResult"]]

In [ ]:
df.fillna("null",inplace=True)

In [ ]:
df

In [ ]:
df = df[df.GoldID != "Mapping_Not_Found"]

In [ ]:
df

In [ ]:
df_group = df.groupby("GoldID")

In [ ]:
m_vs_one_index = []
one_vs_m_index = []
one_vs_one_index = []
for key, value in df_group.groups.items():
    if len(value) > 1:
        m_vs_one_index.extend(value) ## add the index of multiple to one instance to list
    elif "||" in key:
        one_vs_m_index.extend(value)
    else:
        one_vs_one_index.extend(value)

## M vs one

In [ ]:
m_vs_one_df = df[df.index.isin(m_vs_one_index)]

In [ ]:
m_vs_one_df

## one vs M

In [ ]:
one_vs_m_df = df[df.index.isin(one_vs_m_index)]

In [ ]:
one_vs_m_df

## one vs one

In [ ]:
one_vs_one_df = df[df.index.isin(one_vs_one_index)]

In [ ]:
one_vs_one_df

## metrec

In [ ]:
def calculate_acc_at_k(goldId, predId, k):
    correct = 0
    for gold, pred in zip(goldId, predId):
        if any(p in gold for p in pred[:k]):
            correct += 1
    return correct / len(goldId)


def calculate_precision_recall_f1(goldId, predId):
    true_positive = 0
    total_predicted = 0
    total_gold = 0
    
    for gold, pred in zip(goldId, predId):
        true_positive += len(set(gold) & set(pred))
        total_predicted += len(pred)
        total_gold += len(gold)
    
    precision = true_positive / total_predicted if total_predicted > 0 else 0
    recall = true_positive / total_gold if total_gold > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1


def calculate_precision_recall_at_k(goldId, predId, k):
    true_positive = 0
    total_predicted = 0
    total_gold = 0
    
    for gold, pred in zip(goldId, predId):
        true_positive += len(set(gold) & set(pred[:k]))
        total_predicted += min(len(pred[:k]), k)
        total_gold += len(gold)
    
    precision = true_positive / total_predicted if total_predicted > 0 else 0
    recall = true_positive / total_gold if total_gold > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1

## extracting goldid and predid

In [ ]:
## parse json
def get_predId(element):
    predid = element.get("conceptId")
    return predid

In [ ]:
## get the actual results:
def filter_results(results):
    result_list = []
    for result in results:
        if result.get("score") > 0:
            result_list.append(result)
    return result_list

In [ ]:
## extracting goldId and predId
def extract_GoldIdAndPredId(df):
    goldIds = []
    predIds = []
    for row in df.itertuples():
        goldId = row.GoldID.split("||")
        goldIds.append(goldId)
        
        all_results = row.allResult
        if 'null' in all_results:
            all_results = all_results.replace('null', 'None')
        allResults = eval(all_results)[:10]
        allResults = filter_results(allResults)
        predId_list = []
        if len(allResults) == 0:
            predId_list.append("Mapping_Not_Found")
        else:
            for result in allResults:
                predId = get_predId(result)
                predId_list.append(predId)
        predIds.append(predId_list)
    return goldIds, predIds
        

## result of one vs one

In [ ]:
goldIds, predIds = extract_GoldIdAndPredId(one_vs_one_df)

In [ ]:
k_values = [1, 5, 10]

total_acc = {}

for k in k_values:
    avg_acc = calculate_acc_at_k(goldIds, predIds, k)
    precison, recall, f = calculate_precision_recall_at_k(goldIds,predIds,k)
    total_acc[f'acc@{k}'] = avg_acc
    total_acc[f"precision@{k}"] = precison
    total_acc[f"recall@{k}"] = recall
    total_acc[f"f1@{k}"] = f

p, r, f1 = calculate_precision_recall_f1(goldIds,predIds)
total_acc['precision'] = p
total_acc['recall'] = r
total_acc['f1'] = f1

total_acc

## result of M vs one

In [ ]:
goldIds, predIds = extract_GoldIdAndPredId(m_vs_one_df)

In [ ]:
k_values = [1, 5, 10]

total_acc = {}

for k in k_values:
    avg_acc = calculate_acc_at_k(goldIds, predIds, k)
    precison, recall, f = calculate_precision_recall_at_k(goldIds,predIds,k)
    total_acc[f'acc@{k}'] = avg_acc
    total_acc[f"precision@{k}"] = precison
    total_acc[f"recall@{k}"] = recall
    total_acc[f"f1@{k}"] = f

p, r, f1 = calculate_precision_recall_f1(goldIds,predIds)
total_acc['precision'] = p
total_acc['recall'] = r
total_acc['f1'] = f1

total_acc

## result of one vs M

In [ ]:
goldIds, predIds = extract_GoldIdAndPredId(one_vs_m_df)

In [ ]:
k_values = [1, 5, 10]

total_acc = {}

for k in k_values:
    avg_acc = calculate_acc_at_k(goldIds, predIds, k)
    precison, recall, f = calculate_precision_recall_at_k(goldIds,predIds,k)
    total_acc[f'acc@{k}'] = avg_acc
    total_acc[f"precision@{k}"] = precison
    total_acc[f"recall@{k}"] = recall
    total_acc[f"f1@{k}"] = f

p, r, f1 = calculate_precision_recall_f1(goldIds,predIds)
total_acc['precision'] = p
total_acc['recall'] = r
total_acc['f1'] = f1

total_acc